<a href="https://colab.research.google.com/github/leman-cap13/my_projects/blob/main/Real_%26_Fake_News_custom_loop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Upload dataset

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download razanaqvi14/real-and-fake-news

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/real-and-fake-news.zip', 'r')
zip_ref.extractall()


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df_true=pd.read_csv('/content/True.csv')

In [ ]:
df_fake=pd.read_csv('/content/Fake.csv')

In [ ]:
df_true

In [ ]:
df_fake

In [ ]:
df_true.isna().sum()


In [ ]:
df_fake.isna().sum()

In [ ]:
# Hər dataframe-ə label əlavə et:
df_true["label"] = 1  # Real news
df_fake["label"] = 0  # Fake news

In [ ]:
df = pd.concat([df_true, df_fake], ignore_index=True)

In [ ]:
df

In [ ]:
#Data-ı qarışdır (shuffle):
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

frac=1 – bütün datanı al

sample() – random olaraq qarışdır

reset_index(drop=True) – indexləri sıfırdan başlat

In [ ]:
df

In [ ]:
df["content"] = df["title"] + " " + df["text"] # title ve text ikisinide istifade etmek isteyirem deye birlesdirdim
X = df["content"].values
y = df["label"].values

In [ ]:
y

In [ ]:
df["content"].apply(lambda x: len(x.split())).describe()


#Low level api Custom loop

In [ ]:
from transformers import AutoTokenizer,AutoModel
import torch
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
len(X_train)

In [ ]:
len(X_test)

In [ ]:
#tokenize xtrain xtest

def tokenize_text(texts,labels,max_length=128):
  tokens=tokenizer(
      texts,
      padding=True,
      truncation=True,
      max_length=max_length,
      return_tensors='pt'
  )
  tokens['labels']=torch.tensor(labels)
  return tokens


In [ ]:
train_tokens=tokenize_text(X_train.tolist(),y_train.tolist())
val_tokens=tokenize_text(X_test.tolist(),y_test.tolist())

The Hugging Face tokenizer() expects the text input to be one of:

A str → for a single sentence

A List[str] → for a batch of sentences

A List[List[str]] → for tokenized input like [["I", "love", "NLP"], ["Transformers", "are", "great"]]

In [ ]:
train_tokens

In [ ]:
# make train dataset and valid dataset
from torch.utils.data import TensorDataset, DataLoader

train_dataset=TensorDataset(
    train_tokens['input_ids'],
    train_tokens['attention_mask'],
    train_tokens['labels']
                            )

val_dataset=TensorDataset(
    val_tokens['input_ids'],
    val_tokens['attention_mask'],
    val_tokens['labels']
)

In [ ]:
# now we need to craete data loader
from torch.utils.data import DataLoader

train_loader=DataLoader(
    train_dataset,
    batch_size=128,
    shuffle=True
)

val_loader=DataLoader(
    val_dataset,
    batch_size=128,
    shuffle=False
)

shuffle=True → essential for training to break any patterns in data order

shuffle=False → for validation, we keep it consistent

DataLoader as a smart, iterable "batch generator" for your training data.

Instead of giving all data at once to your model, DataLoader:

Divides your dataset into batches

Shuffles data (if told to)

Feeds it to the model one batch at a time during training

Can use parallel workers to speed up data loading

In [ ]:
# make bert model
from transformers import BertForSequenceClassification

model=BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=2,
    output_attentions=False,
    output_hidden_states=False
)


You don’t need to manually define:

Embeddings

Attention

Final linear layer

It’s all handled.

We usually don’t need these values during standard training and evaluation because:( outpu_attention and output_hidden_states FALSE)

They add extra outputs, which consume more memory.

They slow down forward passes.

Most loss and optimization routines don’t use them.

Since your goal is fine-tuning for classification, you just want the logits output (final predictions), so you save resources by keeping them False.

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


In [ ]:
# Now we need to setup  Optimizer, Loss, and Scheduler

loss_fn=torch.nn.CrossEntropyLoss()
optimizer=torch.optim.AdamW(model.parameters(),lr=2e-5)

In [ ]:
#learning rate scheduler
from transformers import get_scheduler

num_epochs=5
num_training_steps=num_epochs*len(train_loader)
lr_scheduler=get_scheduler(
    name='linear',
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps

)

In [ ]:
from accelerate import Accelerator

accelerator=Accelerator()
model, optimizer, train_loader, val_loader, lr_scheduler = accelerator.prepare(
    model, optimizer, train_loader, val_loader, lr_scheduler
)

In [ ]:
# so far we prepare all we need for a custom loop
from tqdm.auto import tqdm

progress_bar=tqdm(range(num_training_steps))

for epoch in range(num_epochs):
  model.train()
  for batch in train_loader:
    input_ids=batch[0].to(device)
    attention_mask=batch[1].to(device)
    labels=batch[2].to(device)
    batch=input_ids,attention_mask,labels
    outputs=model(input_ids=input_ids,attention_mask=attention_mask,labels=labels)
    loss=outputs.loss
    accelerator.backward(loss)

    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
    progress_bar.update(1)

  model.eval()
  preds=[]
  true_labels=[]
  for batch in val_loader:
    input_ids=batch[0].to(device)
    attention_mask=batch[1].to(device)
    labels=batch[2].to(device)

    with torch.no_grad():
          batch=input_ids,attention_mask,labels
          outputs=model(input_ids=input_ids,attention_mask=attention_mask,labels=labels)
    predictions=outputs.logits.argmax(dim=-1)
    preds.extend(predictions.cpu().numpy())
    true_labels.extend(labels.cpu().numpy())



**tqdm** is a Python library that adds a visual progress bar to loops.
When using PyTorch, model and data must both be on the same device (CPU or GPU).
This line (batch={k:v.to(device) for k, v in batch.items()}) moves each tensor (input_ids, attention_mask, labels, etc.) in the batch to device.

batch={
  "input_ids": ...,
  "attention_mask": ...,
  "labels": ...
}

And **batch unpacks it as:
model(input_ids=..., attention_mask=..., labels=...)


optimizer.step() updates the model weights to reduce the loss.
 loss.backward() computes gradients
 optimizer.step() applies them (i.e., makes the model learn)
 optimizer.zero_grad() resets gradients to zero for the next batch

argmax(dim=-1) returns the index of the max logit (i.e., predicted class) for each row.

dim=-1 means apply argmax across the last dimension → across class scores (usually 0: fake, 1: real).

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(true_labels, preds)
print(f"Epoch {epoch + 1}: Validation Accuracy = {accuracy:.4f}")


#Extractor

In [ ]:
#indi mene tokenler elde etmek lazimdi bunun ucun AutoTokenizer isledecem
from transformers import AutoTokenizer, AutoModel
import torch

model_ckpt = 'distilbert-base-uncased'

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = AutoModel.from_pretrained(model_ckpt).to(device)

In [ ]:
# embedding class
def get_embedding(text):
  inputs=tokenizer(text, return_tensors='pt',truncation=True,padding=True,max_length=512)
  inputs = {k: v.to(device) for k, v in inputs.items()}
  with torch.no_grad():
    outputs=model(**inputs)
  cls_embedding=outputs.last_hidden_state[:,0,:]
  return cls_embedding.squeeze().cpu().numpy()

In [ ]:
#embeddinglerini gotur
embeddings = []
for text in df['content']:
    emb = get_embedding(text)
    embeddings.append(emb)

In [ ]:
#X i update et
X=np.array(embeddings)

In [ ]:
X

In [ ]:
y


In [ ]:
#Trian test e bolmek ucun train_test_split istifade etdim
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
from sklearn.linear_model import LogisticRegression
lr_clf=LogisticRegression()
lr_clf.fit(X_train, y_train)

In [ ]:
lr_clf.score(X_test, y_test)

In [ ]:
lr_clf.score(X_train, y_train)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

def plot_confusion_matrix(y_preds, y_true, labels):
  cm=confusion_matrix(y_true, y_preds, normalize='true')
  fig,ax=plt.subplots(figsize=(6,6))
  disp=ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
  disp.plot(cmap='Blues', values_format='.2f', ax=ax, colorbar=False)
  plt.title('Normalized confusion matrix')
  plt.show()
y_preds=lr_clf.predict(X_test)
plot_confusion_matrix(y_preds,y_test, labels=['Real', 'Fake'])

In [ ]:
sample_text = df['content'][0]

# Yenidən tokenləşdir:
inputs = tokenizer(sample_text, return_tensors='pt', truncation=True, padding=True, max_length=512)

# Token id-ləri geri çevirmək:
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
print(tokens)


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_preds))

#fine tuning

In [ ]:
#fine tuning

from transformers import  AutoModelForSequenceClassification
num_labels=2
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels).to(device)

In [ ]:
# metrics hazirlayaq
from sklearn.metrics import accuracy_score, f1_score
def compute_metrics(pred):
  labels=pred.label_ids
  preds=pred.predictions.argmax(-1)
  f1=f1_score(labels, preds, average='weighted')
  acc=accuracy_score(labels, preds)
  return {'accuracy': acc, 'f1': f1}

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from transformers import TrainingArguments

batch_size=64

model_name=f'{model_ckpt}-finetuned-fake_true_news'
training_args=TrainingArguments(
    output_dir=model_name,  #main
    num_train_epochs=2,  #main
    learning_rate=2e-5,   #main
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    eval_strategy='epoch',
    disable_tqdm=False,
    push_to_hub=True,
    log_level='error'
)

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["content"].tolist(),
    df["label"].tolist(),
    test_size=0.2,
    random_state=42
)

In [ ]:
from datasets import Dataset

train_dataset = Dataset.from_dict({
    "text": train_texts,
    "label": train_labels
})

val_dataset = Dataset.from_dict({
    "text": val_texts,
    "label": val_labels
})


In [ ]:
def tokenize_function(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=512
    )

In [ ]:
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

train_dataset = train_dataset.remove_columns(["text"])
val_dataset = val_dataset.remove_columns(["text"])

In [ ]:
from transformers import Trainer

trainer=Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

In [ ]:
trainer.train()

In [ ]:
preds_output=trainer.predict(val_dataset)

In [ ]:
preds_output

In [ ]:
trainer.push_to_hub(commit_message='Training completed')